In [ ]:
import huggingface_hub
hf_token = '' # put your User Access Tokens here
# ابتدا login کنید
huggingface_hub.login(token=hf_token)

# سپس وضعیت ورود را بررسی کنید
!hf auth whoami

user:  AM-Nateghi


In [3]:
# ================================================================
# Text Normalization
# ================================================================
ARABIC_TO_PERSIAN = {"\u064a": "ی", "\u0643": "ک", "\u06cc": "ی"}
PERSIAN_DIGITS_TO_EN = {f"۰۱۲۳۴۵۶۷۸۹"[i]: str(i) for i in range(10)}


def normalize_text(text: str) -> str:
    """Normalize Persian text"""
    if not isinstance(text, str):
        return ""
    for ar, fa in ARABIC_TO_PERSIAN.items():
        text = text.replace(ar, fa)
    for fa, en in PERSIAN_DIGITS_TO_EN.items():
        text = text.replace(fa, en)
    text = " ".join(text.split())
    text = text.replace(" ؟", "؟").replace("،", "، ")
    return " ".join(text.split())

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

torch.cuda.empty_cache()

cptk = "google/gemma-3-270m"

# بارگذاری tokenizer
tokenizer = AutoTokenizer.from_pretrained("outputs/gemma3_safe/final/")
# بارگذاری مدل پایه
base_model = AutoModelForCausalLM.from_pretrained(cptk)
# بارگذاری LoRA روی مدل پایه
model = PeftModel.from_pretrained(base_model, "outputs/gemma3_safe/final/")
# انتقال به GPU
model.to("cuda")
print("ok!")

/home/ai/Desktop/Fine-Tune/.env/lib/python3.12/site-packages/peft/tuners/tuners_utils.py:693: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['model.embed_tokens'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


ok!


In [6]:
q_norm = normalize_text("برام اثبات کن که خدا وجود داره. خیلی خلاصه زود تند سریع.")
text = f"سوال: {q_norm}\nپاسخ: "
output = tokenizer.decode(
    model.generate(
        **tokenizer(text, return_tensors="pt").to(
            "cuda"
        ),
        max_new_tokens=512,
    )[0],
    skip_special_tokens=True
)
print(output)

سوال: برام اثبات کن که خدا وجود داره. خیلی خلاصه زود تند سریع.
پاسخ: ًآیا خب میاد خدا منطقه ی خدا رو درک نمیکنه؟ یا اگه خدا در بین خدا ها داخل عالم واقعا چقدر درک میفهם؟
چرا؟ یا خب میاد دیگه چرا کسی رو درک نمیفهم؟!
مثلا یک بیمار مبتلا به جگر شده در اتاق طباب، پزشک بیمار گفته: چطور این انسان جگر رو در میفته؟ آیا شما هم عینک خودتان رو ببینید؟ خب مردم میگن انسان خدا در همه عالم هستی هست چرا مردم میگن ما در عالم واقع هستی هست؟ فقط خود من میگم خدا عالم واقعیه نه خدا عالم های دیگر میگه عالم واقعیه، عینک من نیست؟ اینطور نیست یا خب نمیشه؟ نمیتونه اینطور نیست نمیتونه خدا عالم نیست یعنی خیلی ها فکر کردن براش نیست میگن دنیا در بین ما نیست. همین سوال ها میگئن؟ چرا وقتی ما کسی را در می فرستم و به خاطر این روز گمشده میفهمیم که عالم ها عالم نیست و در واقع عالم واقعیه هستی؟
چشمگجابند چطوری میبینیم که دنیا عالم نیست؟ تو این خط دفاعی میگند عالم واقعیه نیست که عالم عالم واقعیه نیست وگر بگو خدا در تمام عالم دنیا هست و چرا دنیا عالم نیست؟ چون عالم در بین خدا ها عالم نیست و عالم های دیگری عالم هم هست و عالم